In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import os
import math
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill,Color
from tqdm import tqdm
import shutil

# path
cur_dir = os.path.dirname(os.path.realpath('__file__'))
root = os.path.abspath(os.path.join(cur_dir, os.pardir))
download = os.path.abspath(os.path.join(os.path.abspath(os.path.join(root, os.pardir)), 'Downloads'))
fandango = os.path.join(cur_dir, 'fandango_bak')

print(download, cur_dir, fandango)

if not os.path.exists(fandango):
    os.mkdir(fandango)

In [ ]:
now_date = datetime.today().strftime("%Y-%m-%d")
last_date = datetime.today() - timedelta(days=7)

# 예외적으로 일주일 이내에 작업함
# last_date = datetime.today() - timedelta(days=6)

last_date = last_date.strftime("%Y-%m-%d")

now_date, last_date

In [ ]:
_kind = input("Enter the kind: (1: now, 2: open)")

while True:
    if _kind == '1':
        kind = 'now'
        now = f"now_playing_update_{now_date}.xlsx"
        last = f"now_playing_update_{last_date}.xlsx"
        save = f"(save)now_playing_update_{now_date}.xlsx"
        break
    elif _kind == '2':
        kind = 'open'
        now = f"opening_this_week_{now_date}.xlsx"
        last = f"opening_this_week_{last_date}.xlsx"
        save = f"(save)opening_this_week_{now_date}.xlsx"
        break
    else:
        _kind = input("Enter the kind: (1: now, 2: open)")
        
now, last, save

In [ ]:
# file path
_now_path = os.path.join(download, now)
_last_path = os.path.join(download, last)
now_path = os.path.join(fandango, now)
last_path = os.path.join(fandango, last)
add_path = last_path

save_path = os.path.join(fandango, save)

# move file
if os.path.exists(_now_path):
    shutil.move(_now_path, now_path)
if os.path.exists(_last_path):
    shutil.move(_last_path, last_path)

In [ ]:
res = pd.read_excel(now_path)
res = res.astype({'release_date': 'str'})
res.loc[:, 'release_date'] = res.release_date.str[:10]

last = pd.read_excel(last_path , sheet_name='Sheet2')
last = last.astype({'release_date': 'str'})
last.loc[:, 'release_date'] = last.release_date.str[:10]


res = res.drop(['Unnamed: 0'], axis=1)
last = last.drop(['Unnamed: 0'], axis=1)

#0 : 중복 아님, 1: 저번주것과 중복인데 값 없음, 2: 저번주와 중복인데 값존재  ==> 중복 (1): 노란색 , (0) : 빨간색
res['(yellow)overlap']=0 

#수기 검수 때 편리하게 하기 위한 컬럼, 나중에 보고드릴 땐 삭제하고 드리면 됨
res['fandango'] = ''
res['tmdb'] = ''
res['imdb'] = ''


res['series_id'] = res['series_id'].astype(str) #id 값 문자열로 변환
res['tmdb_id'] = res['tmdb_id'].astype(str)

last['series_id'] = last['series_id'].astype(str)
last['tmdb_id'] = last['tmdb_id'].astype(str)

In [ ]:
# 저번주에 찾은 정보 대입 - sheet2 채운 값들 불러오기

for a in range(0,len(res)):
    res.loc[a, 'series_id']=res.loc[a, 'series_id'].replace('.0','')
    res.loc[a, 'tmdb_id']=res.loc[a, 'tmdb_id'].replace('.0','')
    for b in range(0,len(last)):
        if res.loc[a, 'title_fandango']==last.loc[b, 'title_fandango']:
            if res.loc[a, 'series_id'] =='nan':
                res.loc[a, '(yellow)overlap']=1
            else:
                res.loc[a, '(yellow)overlap']=2  
            break

add1_info = pd.read_excel(add_path , sheet_name='Sheet2')
add1_info = add1_info.astype({'release_date': 'str'})
add1_info.loc[:, 'release_date'] = add1_info.release_date.str[:10]
add1 = add1_info.drop(['Unnamed: 0'], axis=1)

add2_info = pd.read_excel(last_path , sheet_name='Sheet2')
add2_info = add2_info.astype({'release_date': 'str'})
add2_info.loc[:, 'release_date'] = add2_info.release_date.str[:10]
add2 = add2_info.drop(['Unnamed: 0'], axis=1)

for a in range(0,len(add1)):
    add1.loc[a, 'series_id']=str(add1.loc[a, 'series_id']).replace('.0','')
    add1.loc[a, 'tmdb_id']=str(add1.loc[a, 'tmdb_id']).replace('.0','')
    
for a in range(0,len(add2)):
    add2.loc[a, 'series_id']=str(add2.loc[a, 'series_id']).replace('.0','')
    add2.loc[a, 'tmdb_id']=str(add2.loc[a, 'tmdb_id']).replace('.0','')

for a in range(0,len(res)):
    for b in range(0,len(add1)):
        if res.loc[a, 'title_fandango'] == add1.loc[b, 'title_fandango']:
            res.loc[a, 'imdb_id'] = add1.loc[b, 'imdb_id']
            res.loc[a, 'series_id'] = add1.loc[b, 'series_id']
            res.loc[a, 'tmdb_id'] = add1.loc[b, 'tmdb_id']
            res.loc[a, 'title'] = add1.loc[b, 'title']
            res.loc[a, 'is_now_playing'] = add1.loc[b, 'is_now_playing']
            res.loc[a, 'release_date'] = add1.loc[b, 'release_date']
            break

for a in range(0,len(res)):
    for b in range(0,len(add2)):
        if res.loc[a, 'title_fandango'] == add2.loc[b, 'title_fandango']:
            res.loc[a, 'imdb_id'] = add1.loc[b, 'imdb_id']
            res.loc[a, 'series_id'] = add2.loc[b, 'series_id']
            res.loc[a, 'tmdb_id'] = add2.loc[b, 'tmdb_id']
            res.loc[a, 'title'] = add2.loc[b, 'title']
            res.loc[a, 'is_now_playing'] = add2.loc[b, 'is_now_playing']
            res.loc[a, 'release_date'] = add2.loc[b, 'release_date']
            break

In [ ]:
# tmdb 검색시 y:2022 를 추가하면 2022년도에 가까운 순으로 검색 결과가 나옴
# 해당 기능을 이용하기 위한 코드

def tmdb_url_find(url):
    t=' ' 
    year=' '
    for a in range(0,len(url)):
        if url[a] =='(' and url[a+5]==')':
            year = url[a+1:a+5]
            year = 'y:'+year
            break
        else:
            t = t+url[a]
            
    return t+year

# nan 값을 가지고 있는 셀에 사이트 링크 연결 ==> 검색 하는 시간 단축을 위함 , 검색이 안될 수도 있음
for a in range(len(res)):
    url = res.loc[a, 'title_fandango']
    
    # fandango url
    res.loc[a, 'fandango'] = 'https://www.fandango.com'+res.loc[a, 'ticketing_url']
    
    # tmdb url
    tmdb_url = tmdb_url_find(url).replace(' ','%20')
    res.loc[a, 'tmdb'] = 'https://www.themoviedb.org/search/movie?query='+tmdb_url
    
    # imdb url
    imdb_id = res.loc[a, 'imdb_id']
    if imdb_id == "none":
        res.loc[a, 'imdb'] = "https://www.imdb.com/find?q="+url.replace(' ','+')+'+&ref_=nv_sr_sm'
    else:
        res.loc[a, 'imdb'] = "https://www.imdb.com/title/" + imdb_id

In [ ]:
_temp = os.path.join(fandango, 'fandango_mapping_temporary_file_temp.xlsx')
res.to_excel(_temp)
wb = load_workbook(filename=_temp)
ws = wb.active

for a in range(0,len(res)):
    
    if res.loc[a, '(yellow)overlap'] ==0:  
        for col_num in range(1,11):
            color = Color(indexed = 2)
            paint_cell = PatternFill(patternType='solid', fgColor=color)
            
            select_cell = ws.cell(row=a+2, column=col_num)
            select_cell.fill = paint_cell        
    elif res.loc[a, '(yellow)overlap'] ==1:  
        for col_num in range(1,11):
            color = Color(indexed = 5)
            paint_cell = PatternFill(patternType='solid', fgColor=color)
            
            select_cell = ws.cell(row=a+2, column=col_num)
            select_cell.fill = paint_cell
            
ws.delete_cols(11)
wb.save(save_path)

os.remove(_temp)

---
### Mapping: tmdb

In [ ]:
import os
import re
import sys
import time
import json
import requests
# import pickle
import numpy as np
import pandas as pd

# Scrapping
from bs4 import BeautifulSoup
from selenium import webdriver
from user_agent import generate_user_agent
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.alert import Alert
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException, NoSuchElementException, TimeoutException

# Exception Error Handling
import socket
import warnings
warnings.filterwarnings("ignore")

def get_url(url, window=False, image=False):
    ''' Set up webdriver, useragent & Get url '''
    
    wd = None
    socket.setdefaulttimeout(30)
    error = []
    attempts = 0 # url parsing 시도횟수
    # 10번 이상 parsing 실패시 pass
    while attempts < 10:
        try:  
            attempts += 1
            # user agent
            options = Options() 
            userAgent = generate_user_agent(os=('mac', 'linux'), navigator='chrome', device_type='desktop')
            options.add_argument('window-size=1920x1080')
            options.add_argument("--disable-gpu")
            options.add_argument('--disable-extensions')
            options.add_argument(f'user-agent={userAgent}')
            
            if not window:
                options.add_argument('headless')
            if not image:
                options.add_argument('--blink-settings=imagesEnabled=false')

            # web driver 
            wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
            wd.get(url)
            wd.implicitly_wait(5)
            break

        # 예외처리
        except Exception as e:
            print(f'\n\nError: {str(e)}\n\n')
            
            time.sleep(30)
            try:
                wd.quit()
            except:
                pass
            wd = None
    return wd
    
def scroll_down(wd, sleep_time, check_count):
    ''' page scroll down '''
    
    cnt = 0
    while True:
        height = wd.execute_script("return document.body.scrollHeight")
        wd.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(sleep_time)
        cnt += 1
        if cnt == check_count:
            break        
    return wd

def get_headers():
    userAgent = generate_user_agent(os=('mac', 'linux'), navigator='chrome', device_type='desktop')
    headers = {
        "user-agent": userAgent,
        "Accept": "application/json",
    }
    return headers

def json_iterator(url, iterations=5, headers=True):
    
    if headers:
        headers = get_headers()
    else:
        headers = None
    cnt = 0
    res_data = None
    while cnt <= iterations:
        try:
            res = requests.get(url, headers=headers)
            res_data = json.loads(res.text)
            break
        except Exception as e:
            print(f'Error: {e}')
        cnt += 1
        time.sleep(100)
        
    return res_data

In [ ]:
from hangle._distance import jamo_levenshtein

month = {
    'January': '01',
    'February': '02',
    'March': '03',
    'April': '04',
    'May': '05',
    'June': '06',
    'July': '07',
    'August': '08',
    'September': '09',
    'October': '10',
    'November': '11',
    'December': '12',
}

movie_df = res.drop(columns='(yellow)overlap').copy()
movie_df.loc[movie_df.series_id=="nan", ["series_id", "tmdb_id"]] = None, None
for idx in tqdm(range(len(movie_df))):
    title_fandango_org = movie_df.loc[idx, 'title_fandango']
    express = re.compile(r'[\(]+[0-9]+[\)]+')
    title_fandango = re.sub(express, '', title_fandango_org).strip()
    _tmdb_id = movie_df.loc[idx, 'tmdb_id']
    url = movie_df.loc[idx, 'tmdb']

    if _tmdb_id is not None:
        movie_df.loc[idx, ['similarity', 'status_mapping']] = 1, 1
        continue
    else:
        wd = get_url(url, window=False, image=False)
        time.sleep(1.5)
        html = BeautifulSoup(wd.page_source, 'lxml')
        movies = html.find_all('div', 'title')
        status_mapping = 0
        tmdbs, sims = [], []
        mapping = False
        for movie in movies:
            
            # tmdb id
            tmdb_id = movie.find('a')['href']
            if '/movie/' in tmdb_id:
                tmdb_id = tmdb_id.replace('/movie/', '').strip()
            else:
                continue
            
            # release_date
            if movie.find('span') is None:
                release_date = ''    
            else:
                release_date = movie.find('span').text.split()
                if len(release_date) == 3:
                    m = month[release_date[0]]
                    d = release_date[1].replace(',', '')
                    if len(d) == 1:
                        d = '0' + d
                    y = release_date[2]
                    release_date = f'{y}-{m}-{d}'
                else:
                    release_date = '' 
                    
            # title
            if movie.find('a') is None:
                title = ''
            else:
                title_org = movie.find('a').text.strip()
                express = re.compile(r'[\(]+[0-9]+[\)]+')
                title = re.sub(express, '', title_org).strip()

            # Check similarity
            status_mapping = 1
            a = title.replace(' ', '').lower()
            b = title_fandango.replace(' ', '').lower()
            dist = jamo_levenshtein(a, b)
            max_len = max(len(a), len(b))
            sim = (max_len - dist) / max_len
            
            if sim == 1:
                mapping = True
                break
            sims.append(sim)
            print(title_fandango, tmdb_id, title, release_date, sim)
            tmdbs.append([tmdb_id, title, release_date, sim])   
        
        if status_mapping == 0:
            movie_df.loc[idx, ['series_id', 'tmdb_id', 'title', 'release_date', 'similarity', 'status_mapping']] = np.nan, np.nan, np.nan, '', np.nan, status_mapping
        else:
            if mapping:
                max_sim = sim
            else:
                max_sim = max(sims)
                max_idx = sims.index(max_sim)
                max_tmdbs = tmdbs[max_idx]
                tmdb_id, tmdb_id, title, release_date = max_tmdbs[0], max_tmdbs[0], max_tmdbs[1], max_tmdbs[2]
            
            if max_sim == 1:
                # 맵핑 완료
                status_mapping = 1
            elif max_sim >= 0.9:
                # 맵핑 후보 (수기검수 필요)
                status_mapping = 2
            else:
                # 맵핑 안됨 (수기검수 필요)
                status_mapping = 3
                
            movie_df.loc[idx, ['series_id', 'tmdb_id', 'title', 'release_date', 'similarity', 'status_mapping']] = tmdb_id, tmdb_id, title, release_date, max_sim, status_mapping
        
        movie_df.loc[movie_df.release_date=='nan', 'release_date'] = ""
        wd.quit()

In [ ]:
movie_df.info()

In [ ]:
movie_df.groupby('status_mapping').count()

In [ ]:
movie_df.groupby('similarity').count()

---
### Mapping: imdb

In [ ]:
for idx in tqdm(range(len(movie_df))):

    title_fandango_org = movie_df.loc[idx, 'title_fandango']
    express = re.compile(r'[\(]+[0-9]+[\)]+')
    title_fandango = re.sub(express, '', title_fandango_org).strip()
    url = movie_df.loc[idx, 'imdb']
    _imdb_id = movie_df.loc[idx, 'imdb_id']
    if _imdb_id != 'none':
        movie_df.loc[idx, ['similarity_imdb', 'status_mapping_imdb']] = 1, 1
    else:
        wd = get_url(url, window=False, image=False)
        time.sleep(2.5)
        
        html = BeautifulSoup(wd.page_source, 'lxml')
        movies = html.find_all('td', 'result_text')
        if len(movies) == 0:
            movies = html.find_all('div', 'ipc-metadata-list-summary-item__tc')
        
        status_mapping = 0
        imdbs, sims = [], []
        mapping = False
        for movie in movies:
            # imdb_id
            imdb_id = movie.find('a')['href'].split('/')[2]

            # title
            title_imdb = movie.find('a').text.strip()
            express = re.compile(r'[\(]+[0-9]+[\)]+')
            title_imdb = re.sub(express, '', title_imdb).strip()
            
            # Check similarity
            status_mapping = 1
            a = title_imdb.replace(' ', '').lower()
            b = title_fandango.replace(' ', '').lower()
            dist = jamo_levenshtein(a, b)
            max_len = max(len(a), len(b))
            sim = (max_len - dist) / max_len
            
            print(title_fandango, imdb_id, title_imdb, sim)
            if sim == 1:
                mapping = True
                break
            sims.append(sim)
            imdbs.append([imdb_id, title_imdb, sim])


        if status_mapping == 0:
            movie_df.loc[idx, ['imdb_id', 'title_imdb', 'similarity_imdb', 'status_mapping_imdb']] = None, "", np.nan, status_mapping
        else:
            if mapping:
                max_sim = sim
            else:
                max_sim = max(sims)
                max_idx = sims.index(max_sim)
                max_imdbs = imdbs[max_idx]
                imdb_id, title_imdb = max_imdbs[0], max_imdbs[1]
                
            if max_sim == 1:
                # 맵핑 완료
                status_mapping = 1
            elif max_sim >= 0.9:
                # 맵핑 후보 (수기검수 필요)
                status_mapping = 2
            else:
                # 맵핑 안됨 (수기검수 필요)
                status_mapping = 3
                
            movie_df.loc[idx, ['imdb_id', 'title_imdb', 'similarity_imdb', 'status_mapping_imdb']] = imdb_id, title_imdb, max_sim, status_mapping
            
        wd.quit()

In [ ]:
movie_df.groupby("status_mapping_imdb").count()

In [ ]:
movie_df.groupby("similarity_imdb").count()

In [ ]:
drops = ['fandango', 'tmdb', 'imdb', 'similarity', 'status_mapping']

# movie_df_1 = movie_df[(movie_df.status_mapping==1) | (movie_df.status_mapping==2)].drop(columns=drops).reset_index(drop=True)
movie_df_1 = movie_df[(movie_df.status_mapping==1) | (movie_df.status_mapping==2)].reset_index(drop=True)
movie_df_1.loc[:, 'is_now_playing'] = 1

# movie_df_0 = movie_df[(movie_df.status_mapping==0) | (movie_df.status_mapping==3)].drop(columns=drops).reset_index(drop=True)
movie_df_0 = movie_df[(movie_df.status_mapping==0) | (movie_df.status_mapping==3)].reset_index(drop=True)
movie_df_0.loc[:, 'is_now_playing'] = ''

# origin file
_movie_df = pd.read_excel(now_path).iloc[:, 1:]


# Save file
now_path_final = os.path.join(cur_dir, now)
writer = pd.ExcelWriter(now_path_final, engine = 'openpyxl')
_movie_df.to_excel(writer, sheet_name = 'Sheet1')
movie_df_1.to_excel(writer, sheet_name = 'Sheet2')
movie_df_0.to_excel(writer, sheet_name = 'Sheet3')

writer.save()
writer.close()

print(f"Complete save file:  {now_path_final}")

In [ ]:
# # Check page

# for idx in movie_df[movie_df.status_mapping!=1].index:
#     title_fandango = movie_df.loc[idx, 'title_fandango']
#     print(title_fandango)
    
#     url_tm = movie_df.loc[idx, 'tmdb']
#     url_im = movie_df.loc[idx, 'imdb']
#     wd_tm = get_url(url_tm, True, True)
#     wd_im = get_url(url_im, True, True)
    
#     c = input('Close?(Enter the any key): ')
    
#     wd_tm.quit()
#     wd_im.quit()